In [ ]:
"""Classificando as empresas como nos demais arquivos porém, está sendo imputado dados nos atributos faltando"""

In [4]:
#Importando bibliotecas e abrindo arquivos necessários
import pandas as pd
import numpy as np
from Análise_de_Correlação import analiseCorr
from Normalização import normalizaDf
from ImputandoDados import imputaDados
df= pd.read_excel("input_acoes_NotasFundamentei_20200517.xlsx")
abas= pd.ExcelFile("input_planilhaCanalHolder_20200211.xlsx")
MA=abas.parse('Múltiplos Ações')
FA=abas.parse('Fundamentos Ações')

In [5]:
#Fazendo alterações no arquivo, removendo espaços nos nomes das colunas
MA.columns = [str(c).replace(' ', '_') for c in MA.columns]
FA.columns = [str(c).replace(' ', '_') for c in FA.columns]
df.columns = [str(c).replace(' ', '_') for c in df.columns]

In [6]:
#Join das tabelas "Fundamentos ações" e "NotasCanalFundamentei" e um merge entre as duas primeiras abas do arquivo input_planilhaCanalHolder_20200211.xlsx
MA=MA.join(df.set_index("Código_de_Neg."),on="Código_de_Neg.")
uniaoAbas = pd.merge(MA, FA, on="Nome", how='outer', suffixes=('', '_y'))
uniaoAbas = uniaoAbas.drop(list(uniaoAbas.filter(regex='_y$')), axis=1)
#Remoção de uma coluna inútil
uniaoAbas.drop("Unnamed:_0",axis=1,inplace=True)

In [89]:
#Atualização do dataframe, recebendo novos valores para as colunas numéricas, dessa vez, sem atributos nulos
uniaoAbas.replace("-",np.nan,inplace=True)
uniaoAbas[uniaoAbas._get_numeric_data().columns]=imputaDados("median",uniaoAbas)

In [90]:
dfNormalizado=normalizaDf(uniaoAbas)

In [91]:
#Chamando a função principal
analiseCorrelação=analiseCorr("NotaFundamentei20200201",dfNormalizado)[2:]
analiseCorrelação

Valor_de_Mercado                          0.204
Lucro_Líquido                             0.167
2015                                      0.140
Fluxo_de_Caixa_de_Investimentos_(FCI)     0.129
Ativo_Circulante                          0.098
Patrimônio                                0.096
Patrimônio_Líquido                        0.096
Passivo_Circulante                        0.088
Crescimento_Médio_Anual                   0.083
P/VPA                                     0.081
Cotação                                   0.072
Quantidade_de_Negócios                    0.068
Volume_Negociado                          0.067
PSR                                       0.064
ROIC                                      0.042
ROE                                       0.041
Participação_no_IBRX_100                  0.040
Participação_no_Ibovespa                  0.037
EV/EBITDA                                 0.027
Lucro_Operacional_(EBIT)                  0.014
Dividendo_por_Ação                      

In [92]:
def calculoAutomatizadoScore(analise,dFrame):
    count=0
    for peso in analise.values:
        if peso>0.1:
            print(peso)
            print(analise.axes[0][count])                #score+=dFrame[analise.axes[0][count]][linha]*peso
        #if peso<-0.236:
         #   print(peso)
          #  print(analise.axes[0][count])        
        count+=1
    return dFrame
""" Paramêtros:
        analise: Analise de Correlação feita
        dataFrame: DataFrame contendo as empresas a serem rankeadas
    Return: O mesmo DataFrame do parâmetro, agora contendo a coluna Score """

' Paramêtros:\n        analise: Analise de Correlação feita\n        dataFrame: DataFrame contendo as empresas a serem rankeadas\n    Return: O mesmo DataFrame do parâmetro, agora contendo a coluna Score '

In [93]:
#Descobrindo quais são os principais atributos responsáveis pelo aumento de Score das empresas
testeAutomatizado=calculoAutomatizadoScore(analiseCorrelação,dfNormalizado)

0.204
Valor_de_Mercado
0.167
Lucro_Líquido
0.14
2015
0.129
Fluxo_de_Caixa_de_Investimentos_(FCI)


In [94]:
#Criação da coluna "Score", calculada por meio de uma média ponderada com os atributos mais importantes,
#os pesos estão relacionados aos seus respectivos valores na analise de correlação, exemplo: atributos
#com 0,3 na analise terão peso 3 na média ponderada
dfNormalizado["Score"]=((dfNormalizado["Valor_de_Mercado"]*2)+
(dfNormalizado["Lucro_Líquido"]*1)+
#(dfNormalizado["2015"]*1)+
(dfNormalizado["Fluxo_de_Caixa_de_Investimentos_(FCI)"]*1))/4

In [95]:
dfNormalizado["Score"]=((dfNormalizado["Score"]-dfNormalizado["Score"].min())/(dfNormalizado["Score"].max()-dfNormalizado["Score"].min()))*10

In [96]:
dfNormalizado["Score"]=dfNormalizado["Score"].round(1)

In [97]:
dfNormalizado["Nome"]=uniaoAbas["Nome"]
dfNormalizado["Código_de_Neg."]=uniaoAbas["Código_de_Neg."]

In [98]:
dfNormalizado.sort_values("Score",ascending=False)[["Nome","Código_de_Neg.","Score"]].head(5)

,Nome,Código_de_Neg.,Score
266,Petrobras,PETR4,10.0
265,Petrobras,PETR4,10.0
264,Petrobras,PETR3,10.0
263,Petrobras,PETR3,10.0
195,ItauUnibanco,ITUB3,8.2


In [99]:
#No dataFrame, algumas empresas estão duplicadas, nesse novo dataFrame, essas duplicadas estão sendo eliminadas
testeRemovendoDuplicadas=dfNormalizado.drop_duplicates(subset="Nome")

In [100]:
testeRemovendoDuplicadas.sort_values("Score",ascending=False)[["Nome","Código_de_Neg.","Score"]].head(5)

,Nome,Código_de_Neg.,Score
263,Petrobras,PETR3,10.0
194,ItauUnibanco,ITUB3,8.2
65,Bradesco,BBDC3,6.2
31,Ambev S/A,ABEV3,4.8
301,Santander BR,SANB3,3.7


In [101]:
df[df["Código_de_Neg."]=="SANB3"]

,Código_de_Neg.,NotaFundamentei,NotaFundamentei20200201
